# code to register faces

In [ ]:
import os
import cv2
import numpy as np
import pickle
from insightface.app import FaceAnalysis
from numpy.linalg import norm
app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(640, 640))  # ctx_id=-1 for CPU


# Database of known faces and directories for each person
known_faces = {
    "person1": None,
    "person2": None,
}

# Directories containing images for each person
image_dirs = {
    "person1": r"path",
    "person2": r"path",
}

# Path to save/load the embeddings
pickle_file_path = "face_embeddings.pkl"

# Register known faces by averaging embeddings from multiple images
def register_known_faces():
    for name, directory in image_dirs.items():
        if not os.path.exists(directory):
            print(f"Error: Could not find images directory for {name}")
            continue

        embeddings = []
        for img_file in os.listdir(directory):
            img_path = os.path.join(directory, img_file)
            img = cv2.imread(img_path)
            
            if img is None:
                print(f"Error: Could not load image {img_file} for {name}")
                continue

            # Detect face and extract embedding
            faces = app.get(img)
            if faces:
                embeddings.append(faces[0].embedding)
                print(f"Registered {name}'s face embedding from {img_file}.")
            else:
                print(f"No face detected in {img_file} for {name}.")

        # Average the embeddings for robustness
        if embeddings:
            known_faces[name] = np.mean(embeddings, axis=0)
            print(f"Averaged embedding registered for {name} from {len(embeddings)} images.")

    # Save known faces to a pickle file
    with open(pickle_file_path, "wb") as f:
        pickle.dump(known_faces, f)
    print(f"Face embeddings saved to {pickle_file_path}")


# Identify faces in 

# To initially register and save embeddings, uncomment the line below:
register_known_faces()


# code to recognise faces

In [ ]:

import os
import cv2
import numpy as np
import pickle
from insightface.app import FaceAnalysis
from numpy.linalg import norm



app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(640, 640))  # ctx_id=-1 for CPU

pickle_file_path = "face_embeddings.pkl"

def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (norm(embedding1) * norm(embedding2))

def identify_faces_in_image(input_image_path):
    img = cv2.imread(input_image_path)
    if img is None:
        print("Error: Could not load input image.")
        return
    
    faces = app.get(img)
    if not faces:
        print("No faces detected.")
        return

    for i, face in enumerate(faces):
        best_match_name = "Unknown"
        best_similarity = 0.0

        # Compare with known faces
        for name, known_embedding in known_faces.items():
            if known_embedding is None:
                continue
            similarity = cosine_similarity(face.embedding, known_embedding)
            if similarity > best_similarity:
                best_similarity = similarity
                best_match_name = name

        # Define a similarity threshold
        threshold = 0.6
        if best_similarity >= threshold:
            print(f"Face {i+1}: Identified as {best_match_name} (similarity: {best_similarity:.2f})")
            return [best_match_name, best_similarity]

def load_embeddings():
    global known_faces
    if os.path.exists(pickle_file_path):
        with open(pickle_file_path, "rb") as f:
            known_faces = pickle.load(f)
        print(f"Loaded face embeddings from {pickle_file_path}")
    else:
        print(f"No saved embeddings found. Please run register_known_faces to create them.")

# Load known embeddings
load_embeddings()

# Test with a new image
input_image_path = "input1.jpg"  # Path to an image with unknown faces to identify
name = identify_faces_in_image(input_image_path)
print(name)
